### Introducción del proyecto

El proyecto tiene como objetivo desarrollar un modelo predictivo para recomendar el plan adecuado (Smart o Ultra) a los clientes de Megaline, basado en su comportamiento mensual. Utilizando datos como el número de llamadas, mensajes y tráfico de datos, se entrenarán modelos de clasificación como Regresión Logística, Árbol de Decisión y Random Forest. El modelo deberá alcanzar una exactitud mínima del 75% para predecir correctamente el plan, ayudando a la empresa a optimizar su estrategia comercial y mejorar la satisfacción del cliente.

### Desarrollo:
#### Paso 1: Cargar librerías y explorar dataset



In [1]:
#carga de librerías
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score  
from sklearn.model_selection import train_test_split

In [2]:
users = pd.read_csv("/datasets/users_behavior.csv")

In [3]:
# se usan las funciones info y head para inspección de la información  del dataset
users.info()
print(users.head())
print()
#revisión de filas duplicadas
print(users.duplicated().sum())
#revisión valores ausentes 
print(users.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0

0
calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64


In [4]:
# se cambia el tipo de dato a entero
users['is_ultra'] = users['is_ultra'].astype(int)
users['messages'] = users['messages'].astype(int)
users['calls'] = users['calls'].astype(int)

#### Paso 2: Dividir el dataset en conjuntos de entrenamiento, validación y prueba

El conjunto de prueba no existe, por lo que los datos fuente deben dividirse en tres partes: entrenamiento, validación y prueba. Usualmente, el tamaño del conjunto de validación y el de prueba son iguales. Esto da como resultado una proporción de datos fuente de 3:1:1

In [5]:
# se divide el df total en conjunto de prueba y mix (60%-40%)
users_train, users_mix = train_test_split(users, test_size=0.4, random_state=12345)

# se divide el conjunto mix en prueba y validación (50%-50%)
users_valid, users_test = train_test_split(users_mix, test_size=0.5, random_state=12345)

In [6]:
# se separan las características (features) y la variable objetivo (target),se revisan mediante print
features_train = users_train.drop(['is_ultra'], axis=1)
target_train = users_train['is_ultra']
features_valid = users_valid.drop(['is_ultra'], axis=1) 
target_valid = users_valid['is_ultra'] 
features_test = users_test.drop(['is_ultra'], axis=1) 
target_test = users_test['is_ultra']

In [7]:
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


#### Paso 3: Definir y entrenar los modelos

En esta sección, se definen y entrenan tres modelos: Árbol de Decisión, Regresión Logística y Random Forest. El Árbol de Decisión realiza predicciones dividiendo los datos en función de características clave, la Regresión Logística es un modelo lineal utilizado para clasificación binaria y el Random Forest (Bosque aleatorio) es un modelo de ensamblaje que combina varios árboles de decisión para mejorar la precisión. Todos los modelos serán entrenados con los datos de entrenamiento para ajustar sus parámetros y hacer predicciones en el conjunto de prueba.

##### 1. Árbol de Decisión

In [8]:
best_depth = None
best_accuracy = 0

# se prueban diferentes profundidades
for depth in range(1, 5):
        model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
        model.fit(features_train, target_train)

        predictions_valid = model.predict(features_valid)
        # se calcula la exactitud en el conjunto de validación
        accuracy = accuracy_score(target_valid, predictions_valid)

        print("max_depth =", depth, ": ", end='')
        print(accuracy)
        
        # se guarda la mejor puntuación de accuracy y depth en el conjunto de validación
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_depth = depth
            best_model = model # se guarda el mejor modelo
        



max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944


##### 2. Regresión Logística

In [9]:
best_model2 = LogisticRegression(random_state=12345, solver='liblinear') # se usa liblinear ya que es apropiada para problemas binarios y conjuntos de datos pequeños
best_model2.fit(features_train, target_train) 
score_train = best_model2.score(features_train, target_train) # se calcula la puntuación de accuracy en el conjunto de entrenamiento
score_valid = best_model2.score(features_valid, target_valid) # se calcula la puntuación de accuracy en el conjunto de validación

print("Accuracy del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Accuracy del modelo de regresión logística en el conjunto de validación:", score_valid)

Accuracy del modelo de regresión logística en el conjunto de entrenamiento: 0.7505186721991701
Accuracy del modelo de regresión logística en el conjunto de validación: 0.7589424572317263


##### 3. Random Forest

In [10]:
best_score = 0
best_est = 0
best_model3 = None


for est in range(1, 25): # se selecciona el rango del hiperparámetro para n_estimators
    model3 = RandomForestClassifier(random_state=12345, n_estimators=est) # se configura el número de árboles
    model3.fit(features_train, target_train) 
    score = model3.score(features_valid, target_valid) # calcula la puntuación de accuracy en el conjunto de validación
    
    # guarda la mejor puntuación de accuracy y el número de estimadores correspondientes en el conjunto de validación
    if score > best_score:
        best_score = model3.score(features_valid, target_valid) 
        best_est = est 
        best_model3 = model3  # se guarda el mejor modelo



print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))



La exactitud del mejor modelo en el conjunto de validación (n_estimators = 23): 0.7947122861586314


#### Paso 4: Evaluar los modelos en el conjunto de prueba

En esta fase, se evaluará el rendimiento de los modelos entrenados en el conjunto de prueba para medir su capacidad de generalización. Utilizando la exactitud como métrica, se compararán las predicciones de los modelos de Árbol de Decisión, Regresión Logística y Random Forest con las respuestas reales del conjunto de prueba. Esto nos permitirá identificar cuál de los modelos tiene el mejor desempeño en datos no vistos.

In [11]:
# se evalua el modelo de árbol de decisión en el conjunto de prueba
y_test_ad = best_model.predict(features_test)
accuracy_test_dt = accuracy_score(target_test, y_test_ad)
print(f'Exactitud en prueba (Árbol de Decisión): {accuracy_test_dt:.4f}')

# se evalua el modelo de regresión logística en el conjunto de prueba
y_test_rl = best_model2.predict(features_test)
accuracy_test_log_reg = accuracy_score(target_test, y_test_rl)
print(f'Exactitud en prueba (Regresión Logística): {accuracy_test_log_reg:.4f}')

# se evalua el modelo de Random Forest en el conjunto de prueba
y_test_bd = best_model3.predict(features_test)
accuracy_test_rf = accuracy_score(target_test, y_test_bd)
print(f'Exactitud en prueba (Random Forest): {accuracy_test_rf:.4f}')

Exactitud en prueba (Árbol de Decisión): 0.7792
Exactitud en prueba (Regresión Logística): 0.7403
Exactitud en prueba (Random Forest): 0.7807


Random Forest es el modelo que tiene la mejor calidad en términos de exactitud en el conjunto de prueba con un 78.07%. 

#### Paso 5: Tarea adicional  (prueba de cordura al modelo)

Se realizará una validación cruzada con el fin de evaluar el rendimiento del modelo de manera más robusta y confiable. Este proceso divide el conjunto de datos en varios pliegues, entrenando y evaluando el modelo en cada uno de ellos, lo que permite obtener una estimación más precisa de su capacidad de generalización. 

Además, se llevará a cabo una comprobación de sobreajuste comparando la precisión entre el conjunto de entrenamiento y el conjunto de prueba. Esto nos ayudará a detectar si el modelo está ajustándose excesivamente a los datos de entrenamiento (sobreajuste) o si no está capturando bien los patrones en los datos (subajuste). Ambas técnicas son esenciales para garantizar que el modelo sea estable y capaz de generalizar a nuevos datos.

In [12]:
features = users.drop(['is_ultra'], axis=1)
target = users['is_ultra']

# Realizar validación cruzada
cv_scores = cross_val_score(best_model3, features, target, cv=5)
print(f'Puntajes de validación cruzada: {cv_scores}')
print(f'Media de la validación cruzada: {cv_scores.mean():.4f}')

Puntajes de validación cruzada: [0.79782271 0.76205288 0.79937792 0.78693624 0.80218069]
Media de la validación cruzada: 0.7897


La validación cruzada nos indica que el modelo generalmente está funcionando bien y tiene un rendimiento estable en todos los pliegues.

In [13]:
# comparación entre precisión en entrenamiento y prueba
y_train_pred = best_model3.predict(features_train)
accuracy_train_rf = accuracy_score(target_train, y_train_pred)
print(f'Exactitud en prueba (Random Forest): {accuracy_test_rf:.4f}')
print(f'Exactitud en entrenamiento (Random Forest): {accuracy_train_rf:.4f}')

Exactitud en prueba (Random Forest): 0.7807
Exactitud en entrenamiento (Random Forest): 0.9938


La comparación nos indica que el modelo está sobreajustado, lo que significa que el modelo ha "memorizado" demasiado bien los datos de entrenamiento y no está generalizando correctamente a nuevos datos. Según lo investigado, esto suele suceder cuando el modelo es demasiado complejo o no está regularizado adecuadamente. Algunas estrategias que se puede aplicar en el futuro para mejorar el modelo y reducir el ajuste son las siguientes:
1. Reducir la complejidad del modelo
2. Usar Regularización
3. Aumentar los datos de entrenamiento
4. Early Stopping 
5. Entrenar con más datos representativos

### Conclusión

El proyecto logró desarrollar un modelo predictivo eficaz para recomendar el plan adecuado (Smart o Ultra) a los clientes de Megaline, basado en su comportamiento mensual. A través de la implementación y evaluación de tres modelos de clasificación: Árbol de Decisión, Regresión Logística y Random Forest, se logró identificar que el modelo de Random Forest presentó el mejor desempeño con una exactitud del 78.07% en el conjunto de prueba. 

Durante el proceso, se implementaron técnicas como la validación cruzada y la comprobación de sobreajuste, lo que permitió obtener una estimación más robusta del rendimiento del modelo y detectar que el modelo de Random Forest estaba sobreajustado. Este sobreajuste sugiere que el modelo ha memorizado demasiado los datos de entrenamiento, lo que limita su capacidad de generalizar a nuevos datos. Se sugirieron varias estrategias, como reducir la complejidad del modelo y aumentar los datos de entrenamiento, para mejorar la capacidad de generalización en el futuro. En general, el proyecto proporciona una base sólida para la toma de decisiones en Megaline, con la posibilidad de seguir optimizando los modelos para mejorar aún más su precisión y desempeño.
